In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

import src.convert_cxx_2_pyfloats as cxx2py
import src.thermodynamic_eqns as thermoeqns
import src.generic_axfuncs as axfuncs
# import src.volexponential as volexp
# import src.lognormal as lognorm
# import src.plot_lnr_distrib as pltlnr

sol_filename = "../build/sol.csv"
SDsol_filename = "../build/solSD.csv"
setup_filename = "../build/sol_setup.txt"

savedir = "./ipynbplots/"

In [2]:
### read in constants and intial setup from setup .txt file
CONSTS = cxx2py.read_cpp_into_floats(setup_filename, printout=True)[0]
INITS   = cxx2py.inits_dict(CONSTS, printout=True)
MCONSTS = cxx2py.mconsts_dict(CONSTS, printout=True)
nsupers    = int(INITS["nsupers"])                  
VOL = INITS["DROPVOL"]                        



---- constants (CONSTS) from  ../build/sol_setup.txt -----
TEMP_INIT = 273.15
P_INIT = 100000.0
relh_init = 60.0
qc_init = 0.0
SUPERS = 8192.0
NSUPERS = 2000.0
DROPVOL = 1000000.0
COND_TSTEP = 5e-05
COLL_TSTEP = 1.0
nout = 100.0
iW = 0.5
rtol = 1e-06
G = 9.80665
RGAS_UNIV = 8.314462618
MR_WATER = 0.01801528
MR_DRY = 0.0289647
LATENT_V = 2437300.0
CP_DRY = 1004.9
CP_V = 1864.0
C_L = 4180.0
RHO_DRY = 1.177
RHO_L = 1000.0
le = 1000000.0
RHO_SOL = 2200.0
MR_SOL = 0.058443
IONIC = 2.0
W0 = 0.5
TIME0 = 4000.0
P0 = 100000.0
TEMP0 = 273.15
R0 = 1e-06
---------------------------------------------

---- not floats from  ../build/sol_setup.txt -----
iRho_l = dlc::Rho_l
iRho_sol = dlc::Rho_sol
iMr_sol = dlc::Mr_sol
iIONIC = dlc::IONIC
TSPAN[2] = {0, 4000}
atols[4] = {1e-6, 1e-6, 1e-6, 1e-6}
RGAS_DRY = RGAS_UNIV / MR_DRY
RGAS_V = RGAS_UNIV / MR_WATER
DYNVISC = 18.45 * 1e-6
LATENT_RGAS_V = LATENT_V / RGAS_V
RHO0 = P0 / (DC::RGAS_DRY * TEMP0)
CP0 = DC::CP_DRY
MR0 = DC::MR_DRY
F0 = TIME0 / (RHO0 * R0

In [ ]:
#### Load data from .csv files ###
with open(sol_filename) as file_name:
    t, p, temp, qv, qc = np.loadtxt(file_name, delimiter=",", comments="/*", unpack=True)

with open(SDsol_filename) as file_name:
    drops = np.loadtxt(file_name, delimiter=",", comments="/*", unpack=False)
eps = drops[:,0:nsupers]
r = drops[:,nsupers:2*nsupers]
m_sol = drops[:,2*nsupers:]

print("--- Data Shapes ---")
print(t.shape, temp.shape, p.shape, qv.shape, qc.shape)
print(eps.shape, r.shape, m_sol.shape)
print("\n--- Non Dimensional Max/Mins of Data ---")
print("time:", np.amax(t), np.amin(t))
print("p:", np.amax(p), np.amin(p))
print("temp:", np.amax(temp), np.amin(temp))
print("qv, qc", (np.amax(qv), np.amax(qc)), (np.amin(qv), np.amin(qc)))
print("droplet r:", np.amax(r), np.amin(r))



In [ ]:
#### Get data and Re-Dimensionalise Solution ###
SDprops = CommonSuperdropProperties(nsupers, VOL, CONSTS["RHO_L"], CONSTS["RHO_SOL"], 
                CONSTS["MR_SOL"], CONSTS["IONIC"])
w = INITS["iW"]
t, p, temp, qv, qc = get_soldata(sol_filename, CONSTS["TIME0"], CONSTS["P0"], CONSTS["TEMP0"]) 
z = t*w
eps, r, m_sol = get_SDdata(SDsol_filename, nsupers, CONSTS["R0"], MCONSTS["RHO0"]) 


### Some derived quantities
theta = dry_pot_temp(temp, p, qv)                      # parcel potential temp
gamma = (MCONSTS["RGAS_DRY"]/CONSTS["CP_DRY"]) 
dry_adia = temp[0]*(p/p[0])**gamma            # dry adiabatic temp
dry_adia_theta = dry_adia*(p[0]/p)**gamma    # dry adiabatic theta (=const)

pv = qv*p/(MCONSTS["Mr_ratio"] + qv) 
psat = saturation_pressure(temp)
relh = pv/psat
qsat = MCONSTS["Mr_ratio"] * psat/(p-pv) 
s = qv/qsat - 1   

### cloud base and max supersaturation
basei = np.argmin(abs(s-0))
maxi = np.argmax(s)
cloud_base = z[basei]
cloud_max = z[maxi]
print('\ncloud base at {:.2f}m, s={:.4f}'.format(cloud_base, s[basei]))
print('cloud max at {:.2f}m, s={:.4f}'.format(cloud_max, s[maxi]))
print('cloud max is {:.2f}m above base\n'.format(cloud_max-cloud_base))  



In [ ]:
# plots of time evolution of water as parcel rises

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12,6))
axs = axs.flatten()

axplt(axs[0], t, p/100, ylab='Pressure /mbar', c=0)
axplt(axs[1], t, dry_adia, lab='dry adiabat', c=2, l='--')
axplt(axs[1], t, temp, ylab='T /K', lab='temp', c=3)

axplt(axs[2], t, qv+qc, lab='total', c=7, l='--')
axplt(axs[2], t, qv, lab='vapour', c=5)
axplt(axs[2], t, qc, xlab="time /s", 
    ylab='Water Content', lab='liquid', c=6)

ax3b = axs[3].twinx()
axplt(ax3b, t, s, ylab="supersaturation", c=4)
axplt(axs[3], t, relh, 
    xlab="time /s", ylab='relative humidity', c=4)

for ax in axs:
  ax.legend(fontsize=11, loc='upper right')
axs[0].set_yscale("log")
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(15,6))
axs = axs.flatten()

axplt(axs[0],  z/1000, p/100, 'z=w*t /km', 'P /mbar', c=0)
axplt(axs[1], dry_adia, p/100, lab='dry adiabat', c=2, l='--')
axplt(axs[1], temp, p/100, 'T /K', lab='temp', c=3)
axplt(axs[2], dry_adia_theta, p/100, lab='dry adiabat', c=2, l='--')
axplt(axs[2], theta, p/100, '\u03B8 /K', lab='\u03B8', c=1)

axplt(axs[4], qv+qc, p/100, lab='total', c=7, l='--')
axplt(axs[4], qc, p/100, lab='liquid', c=6)
axplt(axs[4], qv, p/100, 'Water Content', lab='vapour', c=5)


axplt(axs[3], psat/100, p/100, lab='Saturation P', c=2, l='--')
axplt(axs[3], pv/100, p/100, 
        'Water Vapour Pressure /mbar','Vapour P', c=4)

ax5b = axs[5].twiny()
axplt(ax5b, pv/psat, p/100, 'Relative Humidity', c=4)
axplt(axs[5], s, p/100, 'Supersaturation' , c=4)

for ax in axs:
    ax.legend()
    ax.set_yscale('log')
    ax.invert_yaxis()

fig.tight_layout()
plt.savefig(savedir+"solution.png", dpi=400, bbox_inches="tight", facecolor='w')
#plt.close()


In [ ]:
### Moist Static Energy Conservation
mse = moist_static_energy(z, temp, qv)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,4))

axplt(ax, t, mse, xlab ="time /s", 
  ylab='moist static energy / J/Kgm$^{-3}$', c="k")
axb = ax.twinx()
axplt(axb, t, ((mse-mse[0])/mse[0]*100), 
  ylab='% change in moist static energy', c="k")

plt.tight_layout()
plt.show()

In [ ]:
### plots of droplet radii growth
a=1
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
labs = ['{:.2g}\u03BCm'.format(r*1e6) for r in r[:,0]]
for i in range(nsupers)[::a]:
    label = None
    if i == 0:
        label = 'min r0 = '+labs[i]
    elif i == nsupers-1:
        label = 'max r0 = '+labs[i]
    ax1.plot(t, r[:,i]*1e6, linewidth=0.8, label=label)
ax1.set_xlabel('time /s')
ax1.set_ylabel('droplet radius /\u03BCm')
ax1.set_yscale('log')
ax1.legend(fontsize=13)

for i in range(nsupers)[::a]:    
    label = None
    if i == 0:
        label = 'min r0 = '+labs[i]
    elif i == nsupers-1:
        label = 'max r0 = '+labs[i]
    ax2.plot(t, r[:,i]/r[0,i], linewidth=0.8, label=label) #,color=cols[i])
ax2.set_xlabel('time /s')
ax2.set_ylabel('droplet radius / initial radius')
ax2.set_yscale('log')
ax2.legend(fontsize=13)

plt.tight_layout()
plt.savefig(savedir+"SDsolution.png", dpi=400, bbox_inches="tight", facecolor='w')
plt.show()

In [ ]:
lim1 = 0#basei-10
lim2 = -1#np.argmin(abs(z-cloud_base-400))
### plots of droplet radii growth
fig, ax = plt.subplots(figsize=(8,6))
labs = ['{:.2g}\u03BCm'.format(r*1e6) for r in r[:,0]]
for i in range(nsupers):
    label = None
    if i == 0:
        label = 'min r0 = '+labs[i]
    elif i == nsupers-1:
        label = 'max r0 = '+labs[i]
    ax.plot((r[:,i]*1e6)[lim1:lim2], z[lim1:lim2]/1000, linewidth=0.8, label=label)

axb = ax.twiny()
axb.plot(s[lim1:lim2], z[lim1:lim2]/1000, 
         linewidth=1.5, color='k', label='s')
axb.hlines(z[basei]/1000, np.amin(s[lim1:lim2]), np.amax(s[lim1:lim2]), 
         color='gray', linestyle='--', label='cloud base')

ax.set_ylabel('height, z /km')
ax.set_xlabel('droplet radius /\u03BCm')
axb.set_xlabel('supersaturation, s')
ax.set_xscale('log')
ax.legend(fontsize=13, loc='upper left')
axb.legend(fontsize=13,  loc='upper right')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(9,5))
nplts = 10
nbins = 200
rspan = [np.amin(r), np.amax(r)]
tplt = np.linspace(t[0], t[-1], nplts)
#tplt = np.linspace(1650, 1750, nplts)
linear_twinax(ax, np.log(r[0,:]), eps[0,:]/VOL)
for i in range(nplts):
    ind = np.argmin(abs(t-tplt[i]))
    tlab = 't = {:.2f}s'.format(t[ind])
    c = 'C'+str(i)
    logr_distribution(rspan, nbins, r[ind,:], eps[ind,:]/VOL, ax=ax,
        step=True, lab=tlab, c='C'+str(i), ylab="Real Droplet Conc. /m$^{-3}$")
#ax.set_yscale('log')
ax.legend()
plt.tight_layout()

In [ ]:
nplts = 10
nbins = 200
rspan = [np.amin(r), np.amax(r)]
tplt = np.linspace(t[0], t[-1], nplts)
#tplt = np.linspace(1650, 1750, nplts)
ncols = 5
hist0, hedgs0 = logr_distribution(rspan, nbins, r[0,:], eps[0,:]/VOL)

gd = dict(width_ratios=[1]*ncols, height_ratios=[8,2,1,8,2],)
fig, axs = plt.subplots(ncols=5, nrows=5, 
            figsize=(20,12), gridspec_kw=gd, sharex=True)


def histogram_subplot(axa, tplt, c, rspan, nbins, t, r, eps):   
    
    ind = np.argmin(abs(t-tplt))
    tlab = 't = {:.2f}s'.format(t[ind])
    hist, hedgs = logr_distribution(rspan, nbins, r[ind,:], eps[ind,:]/VOL, ax=axa,
        lab=tlab, c=c, ylab="Real Droplet Conc. /m$^{-3}$")
    axa.set_yscale('log') 
    axa.set_xlabel(None)
    
    return hist, hedgs


def barchart_subplot(axa, axb, cens, bars, wdths, rspan, c):
    
    axb.bar(cens+np.log(1e6), bars, wdths, color=c) 
    axb.hlines(0, np.log(rspan[0])+np.log(1e6),
                        np.log(rspan[-1])+np.log(1e6), linestyle='--', linewidth=0.8)
    axb.set_xlabel('ln(r /\u03BCm)')
    axb.set_ylabel('\u0394')

for ax in axs[2,:]:
    ax.set_axis_off()

i = 0
ylimsa = [np.amin(hist0[hist0!=0]), np.amax(hist0)]
for rw in [0,3]:
    for col in range(ncols):
        axa = axs[rw, col]
        axb = axs[rw+1, col]

        hist, hedgs = histogram_subplot(axa, tplt[i], 
                    'C'+str(i), rspan, nbins, t, r, eps) 
        linear_twinax(axa, np.log(r[0,:]), eps[0,:])
        
        hcens = (hedgs[1:]+hedgs[:-1])/2                     
        hwdths = hedgs[1:]- hedgs[:-1]                                         
        barchart_subplot(axa, axb, hcens, hist-hist0, hwdths, rspan, 'C'+str(i))
        
        if np.amin(hist) < ylimsa[0]:
            ylimsa[0] = np.amin(hist[hist!=0])
        if np.amax(hist) > ylimsa[1]:
            ylimsa[1] = np.amax(hist)
        
        i+=1

for rw in [0,3]:
    for col in range(ncols):
        axs[rw, col].set_ylim(ylimsa)
#         axs[rw, col].set_yscale("linear")
    

plt.tight_layout()
#plt.savefig(savedir+"SDdistribution.png", dpi=400, bbox_inches="tight", facecolor='w')

In [ ]:
### plots of droplet epsilon changes
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
labs = ['{:.2g}\u03BCm'.format(r*1e6) for r in r[:,0]]
for i in range(nsupers):
    label = None
    if i == 0:
        label = 'min r0 = '+labs[i]
    elif i == nsupers-1:
        label = 'max r0 = '+labs[i]
    ax1.plot(t, eps[:,i], linewidth=0.8, label=label)
ax1.set_xlabel('time /s')
ax1.set_ylabel('Multiplicity')
#ax1.set_yscale('log')
ax1.legend(fontsize=13)

for i in range(nsupers):    
    label = None
    if i == 0:
        label = 'min r0 = '+labs[i]
    elif i == nsupers-1:
        label = 'max r0 = '+labs[i]
    ax2.plot(t, eps[:,i]/eps[0,i], linewidth=0.8, label=label) #,color=cols[i])

ax2.set_xlabel('time /s')
ax2.set_ylabel('Multiplicity / initial')
#ax2.set_yscale('log')
ax2.legend(fontsize=13)

plt.tight_layout()
plt.show()